In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch
from transformers import GPTNeoForSequenceClassification, GPTNeoForCausalLM, GPT2Tokenizer
from model import GPTNeoForSequenceClassificationBinary
import lightning.pytorch as pl
from model import GPTSequenceClassifiationModule
model = GPTSequenceClassifiationModule.load_from_checkpoint("/root/autodl-tmp/runs/finetune.ckpt")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.1.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.23.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.3.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'score.weight', 'transformer.h.17.attn.attention.bias', 'transformer.h.5.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Changing layer  23
Changing layer  22
Changing layer  21
Changing layer  20


In [2]:
model=model.eval()
model=model.cuda()

In [3]:
test_name='/root/Taxonomy-GPT/data/wordnet_all_false_july4_test.csv'
actual_tree='/root/Taxonomy-GPT/data/bansal_wordnet_true_pairs.csv'

In [4]:
from dataset import get_taxonomy_dataset, get_taxonomy_dataset_binary
dataset = get_taxonomy_dataset_binary(test_name, entire_dataset=True,remove_columns=True)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow


In [5]:
dataset

{'test': Dataset({
     features: ['flag', 'input_ids', 'labels'],
     num_rows: 53968
 })}

In [6]:
def format_dataset(example,tokenizer):
    prompt_template = "I am doing the taxonomy research. I think {child} is a subtopic of {parent}"
    prompt = prompt_template.format(child=example['child'], parent=example['parent'])
    example['input_ids'] = tokenizer.encode(prompt)
    return example
def test_dataloader(str_path,batch_size):
    dataset= get_taxonomy_dataset_binary(str_path,entire_dataset=True)
    dataset = dataset['test']
    dataset = dataset.map(lambda example: format_dataset(example,tokenizer))
    dataset = dataset.remove_columns(['child', 'parent', 'group'])

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(tokenizer),
        num_workers=4,
    )

In [7]:
from torch.utils.data import DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
from tqdm.notebook import tqdm
import torch 

results = []
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer)

testdataloader = test_dataloader(test_name, 64)
with torch.no_grad():
    for step, batch in enumerate(tqdm(testdataloader)):
        batch = batch.to('cuda')
        output = model(batch['input_ids'], batch['attention_mask'])
        predicted_labels = torch.sigmoid(output)
        results.extend(predicted_labels.cpu().detach().numpy().tolist())

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-67b183b76ee90bd1.arrow


  0%|          | 0/844 [00:00<?, ?it/s]

In [8]:
import pandas as pd
df = pd.read_csv(test_name)
df['predict'] = results
df['pred_flag']=[True if i>0.5 else False for i in df['predict']]

In [9]:
df.to_csv('/root/Taxonomy-GPT/data/test_all_res_july_05_3.csv', index=False)

In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test=df['flag']
y_pred=df['pred_flag']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(accuracy_score(y_test, y_pred))

0.6733968649157418
0.6637308498838642
0.684547219833545
0.9456344500444708


In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.97      0.97     51767
        True       0.35      0.40      0.38      2201

    accuracy                           0.95     53968
   macro avg       0.66      0.68      0.67     53968
weighted avg       0.95      0.95      0.95     53968



maximum spanning tree

In [12]:
df

,parent,child,group,flag,predict,pred_flag
0,explosion,blowback,647,True,0.172217,False
1,explosion,blast,647,True,0.378819,False
2,explosion,bomb_blast,647,False,0.715731,True
3,explosion,airburst,647,True,0.711709,True
4,explosion,big_bang,647,True,0.424649,False
...,...,...,...,...,...,...
53963,linolenic_acid,oleic_acid,760,False,0.087052,False
53964,linolenic_acid,linoleic_acid,760,False,0.105808,False
53965,linolenic_acid,docosahexaenoic_acid,760,False,0.092627,False
53966,linolenic_acid,eicosapentaenoic_acid,760,False,0.102251,False


In [13]:
df_t=df

def convert_to_ancestor_graph(G):
    '''Converts a (parent) tree to a graph with edges for all ancestor relations in the tree.'''
    G_anc = nx.DiGraph()
    for node in G.nodes():
        for anc in nx.ancestors(G, node):
            G_anc.add_edge(anc, node)
    return G_anc

In [14]:
import networkx as nx

forest=[]
for g in tqdm(list(set(df_t.group))):
    df_tree=df_t[df_t.group==g]
    graph = nx.DiGraph()
    parents=df_tree['parent'].tolist()
    children=df_tree['child'].tolist()
    probabilities=df_tree['predict'].tolist()
    nodes = set(parents + children)
    for node in nodes:
        graph.add_node(node)
    for i in range(len(parents)):
        graph.add_edge(parents[i], children[i], weight=probabilities[i])
    # print(nx.is_connected(graph))
    T=nx.maximum_spanning_arborescence(graph)
    T = convert_to_ancestor_graph(T)
    df=nx.to_pandas_edgelist(T)
    df['group']=g
    forest.append(df)

  0%|          | 0/114 [00:00<?, ?it/s]

In [15]:
res_v2=pd.concat(forest, ignore_index=True)

In [16]:
res_v2

,source,target,group
0,fragmentation,big_bang,647
1,fragmentation,blowback,647
2,fragmentation,blast,647
3,fragmentation,explosion,647
4,fragmentation,airburst,647
...,...,...,...
3366,unsaturated_fatty_acid,oleic_acid,760
3367,unsaturated_fatty_acid,ricinoleic_acid,760
3368,unsaturated_fatty_acid,docosahexaenoic_acid,760
3369,unsaturated_fatty_acid,linolenic_acid,760


In [17]:
res_v2.columns=['parent','child','group']
res_v2['compare']=res_v2['parent']+res_v2['child']+res_v2['group'].astype(str)
res_v2['group'].value_counts()

group
677    113
712    112
651     79
741     71
666     67
      ... 
743     11
750     11
659     10
682     10
746     10
Name: count, Length: 114, dtype: int64

In [18]:
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['group']>= 647]
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)

In [19]:
forest=[]
for g in tqdm(list(set(df_actual.group))):
    df_tree=df_actual[df_actual.group==g]
    graph = nx.DiGraph()
    parents=df_tree['parent'].tolist()
    children=df_tree['child'].tolist()
    nodes = set(parents + children)
    for node in nodes:
        graph.add_node(node)
    for i in range(len(parents)):
        graph.add_edge(parents[i], children[i])
    # print(nx.is_connected(graph))
    T = convert_to_ancestor_graph(graph)
    df=nx.to_pandas_edgelist(T)
    df['group']=g
    forest.append(df)

df_actual =pd.concat(forest, ignore_index=True)
df_actual.columns=['parent','child','group']
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)
df_actual['group'].value_counts()

  0%|          | 0/114 [00:00<?, ?it/s]

group
677    87
666    87
758    78
741    76
691    76
       ..
746    15
682    14
647    14
737    13
661    13
Name: count, Length: 114, dtype: int64

In [20]:
df_actual[df_actual.group==647]

,parent,child,group,compare
0,explosion,big_bang,647,explosionbig_bang647
1,explosion,blowback,647,explosionblowback647
2,explosion,blast,647,explosionblast647
3,explosion,fragmentation,647,explosionfragmentation647
4,explosion,airburst,647,explosionairburst647
5,explosion,backfire,647,explosionbackfire647
6,explosion,nuclear_explosion,647,explosionnuclear_explosion647
7,explosion,bomb_blast,647,explosionbomb_blast647
8,explosion,inflation,647,explosioninflation647
9,explosion,backblast,647,explosionbackblast647


In [21]:
res_v2[res_v2.group==647]

,parent,child,group,compare
0,fragmentation,big_bang,647,fragmentationbig_bang647
1,fragmentation,blowback,647,fragmentationblowback647
2,fragmentation,blast,647,fragmentationblast647
3,fragmentation,explosion,647,fragmentationexplosion647
4,fragmentation,airburst,647,fragmentationairburst647
5,fragmentation,backfire,647,fragmentationbackfire647
6,fragmentation,nuclear_explosion,647,fragmentationnuclear_explosion647
7,fragmentation,bomb_blast,647,fragmentationbomb_blast647
8,fragmentation,inflation,647,fragmentationinflation647
9,fragmentation,backblast,647,fragmentationbackblast647


In [22]:
df_actual.merge(res_v2, on='compare',how='left').to_csv('/root/Taxonomy-GPT/data/group_wise_july_05.csv')

In [23]:
len(df_actual.merge(res_v2, on='compare'))/len(df_actual)

0.49665095509799057

In [24]:
len(df_actual.merge(res_v2, on='compare'))/len(res_v2)

0.5938890536932661

In [28]:
recall = []
precision = []
f1 = []
for group in tqdm(list(set(df_actual.group))):
    group_actual = df_actual[df_actual.group == group]
    group_pred = res_v2[res_v2.group == group]
    recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))
    precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred))
    if precision[-1] + recall[-1] == 0:
        f1.append(0)
    else:
        f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))

  0%|          | 0/114 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   group_pred = res_v2[res_v2.group == group]                                              │
│    7 │   recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))    │
│    8 │   precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred)    │
│ ❱  9 │   f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))              │
│   10                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ZeroDivisionError: float division by zero

In [26]:
import numpy as np
np.mean(recall)

0.5019478404829713

In [27]:
np.mean(precision)

0.6303051739613261